In [1]:
!pip install pandas


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 1.6 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 85.3 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.2/509.2 kB 82.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 347.8/347.8 kB 49.7 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.3.1 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip


In [2]:
! pip install transformers[torch]


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.3/365.3 kB 7.8 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.3/515.3 kB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.7/781.7 kB 65.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 60.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 65.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 94.4 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.6/199.6 kB 52.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 114.1 MB/s eta 0:00:00
  Attempting unins

In [3]:
import pandas as pd
import re

df_train = pd.read_csv('./train.csv', encoding='utf-8-sig')

df_train['paragraphs'] = df_train['full_text'].apply(
    lambda text: [p.strip() for p in re.split(r'\n+', text) if p.strip()]
)

df_exploded = df_train.explode('paragraphs').reset_index(drop=True)

df_exploded['paragraph_index'] = df_exploded.groupby('title').cumcount()
df_exploded.rename(columns={'paragraphs': 'paragraph_text'}, inplace=True)

par_df = df_exploded[['title', 'paragraph_index', 'paragraph_text', 'generated']]

print(par_df.head())


    title  paragraph_index                                     paragraph_text  \
0  카호올라웨섬                0  카호올라웨섬은 하와이 제도를 구성하는 8개의 화산섬 가운데 하나로 면적은 115.5...   
1  카호올라웨섬                1  마우이섬에서 남서쪽으로 약 11km 정도 떨어진 곳에 위치하며 라나이섬의 남동쪽에 ...   
2  카호올라웨섬                2  1000년경부터 사람이 거주했으며 해안 지대에는 소규모 임시 어촌이 형성되었다. 섬...   
3  카호올라웨섬                3  1830년대에는 하와이 왕국의 카메하메하 3세 국왕에 의해 남자 죄수들의 유형지로 ...   
4  카호올라웨섬                4  1910년부터 1918년까지 하와이 준주가 섬의 원래 모습을 복원하기 위해 이 섬을...   

   generated  
0          0  
1          0  
2          0  
3          0  
4          0  


In [4]:
print(par_df['generated'].value_counts())

generated
0    1125652
1     100712
Name: count, dtype: int64


In [5]:
df_majority = par_df[par_df['generated'] == 0]

df_minority = par_df[par_df['generated'] == 1]


n_minority = len(df_minority)

df_majority_undersampled = df_majority.sample(n=n_minority, random_state=42)



df_undersampled = pd.concat([df_majority_undersampled, df_minority])


print(df_undersampled['generated'].value_counts())



df_undersampled = df_undersampled.sample(frac=1, random_state=42).reset_index(drop=True)


print(df_undersampled.head()) 

generated
0    100712
1    100712
Name: count, dtype: int64
       title  paragraph_index  \
0       몰리에르               37   
1      은진 송씨                3   
2    리오나 루이스               10   
3      진교의 변                3   
4  메흐디 마다비키아                1   

                                      paragraph_text  generated  
0  마들렌의 동생 아르망드와 결혼하여 그 관계에 수수께끼를 남겼다. 부부 사이의 두 아...          1  
1  송유는 12세에 부사정(副司正)이 되었는데, 13세에 관직을 버리고 고향 회덕으로 ...          0  
2  그녀의 음악 캐리어는 더이상 길을 찾지 못할 듯했다. 가수의 꿈을 포기하고 일반 대...          1  
3  조광윤의 군대에 맞서서 최후까지 항전했던 한통(韓通)은 전사했고, 당시 후주의 재상...          0  
4  이란의 페르세폴리스 FC에서 선수 생활을 시작하였다. 1997년 아시아 최우수 청소...          0  


In [1]:
import pandas as pd
import re

a = pd.read_csv('train_undersampled.csv')
a.head()

,title,paragraph_index,paragraph_text,generated
0,몰리에르,37,마들렌의 동생 아르망드와 결혼하여 그 관계에 수수께끼를 남겼다. 부부 사이의 두 아...,1
1,은진 송씨,3,"송유는 12세에 부사정(副司正)이 되었는데, 13세에 관직을 버리고 고향 회덕으로 ...",0
2,리오나 루이스,10,그녀의 음악 캐리어는 더이상 길을 찾지 못할 듯했다. 가수의 꿈을 포기하고 일반 대...,1
3,진교의 변,3,"조광윤의 군대에 맞서서 최후까지 항전했던 한통(韓通)은 전사했고, 당시 후주의 재상...",0
4,메흐디 마다비키아,1,이란의 페르세폴리스 FC에서 선수 생활을 시작하였다. 1997년 아시아 최우수 청소...,0


In [12]:
import pandas as pd
from transformers import AutoTokenizer


try:
    df = pd.read_csv('train_undersampled.csv')
except FileNotFoundError:
    print("오류: 'train_undersampled.csv' 파일을 찾을 수 없습니다.")
    exit()

tokenizer_name = "team-lucid/deberta-v3-base-korean"

try:
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
except ImportError:
    print("오류: 'transformers' 또는 'torch' 라이브러리가 설치되지 않았습니다.")
    print("터미널에서 'pip install transformers torch'를 실행하여 설치하세요.")
    exit()



df['token_count'] = df['paragraph_text'].apply(
    lambda text: len(tokenizer.encode(str(text)))
)


print("--- 'paragraph_text'의 토큰 개수 4분위수 ---")
print(df['token_count'].describe())

--- 'paragraph_text'의 토큰 개수 4분위수 ---
count    201424.000000
mean         82.426975
std          65.214883
min           4.000000
25%          37.000000
50%          68.000000
75%         110.000000
max        3734.000000
Name: token_count, dtype: float64
